In [22]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from openai import OpenAI
from dotenv import load_dotenv
import os
import time

In [23]:
load_dotenv()


True

In [24]:
client = OpenAI(api_key=os.getenv("OPEN_AI_API"))


In [47]:
csv_file_path = f'C:/Users/mkath/Downloads/csv_data/age_gender.csv'  # Replace with the path of'C:\Users\mkath\Downloads\archive (6)'

In [26]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)


In [27]:
description= "You are a Data Distribution Analysis Agent. Your task is to analyze a CSV dataset and determine whether it is fairly and equally distributed across categorical columns.\n\nFollow these instructions strictly:\n1. Remove any rows with null or missing values.\n2. Remove outliers from numerical columns using the IQR method (Q1 - 1.5*IQR to Q3 + 1.5*IQR).\n3. Detect all categorical columns (columns with repeated non-numeric values).\n4. For each categorical column:\n   - Count value occurrences and calculate the percentage distribution.\n   - Identify any categories with fewer than 10% representation or completely missing.\n5. Determine whether each categorical column is equally distributed based on:\n   - No class should dominate over 70%\n   - No class should be under 10% or absent\n\nThen return a structured JSON in this format:\n\n{\n  \"visualize\": {\n    \"column_name\": {\n      \"category1\": percent_value,\n      \"category2\": percent_value,\n      ...\n    },\n    ...\n  },\n  \"env\": {\n    \"status\": \"Equally distributed\" or \"Not equally distributed\",\n    \"missing_or_underrepresented\": [list any underrepresented or missing groups]\n  },\n  \"reason\": {\n    \"verdict\": \"short summary of imbalance or fairness\",\n    \"suggestions\": [\n      \"suggest how to improve representation\",\n      ...\n    ]\n  }\n}\n\nKeep the response clean and strictly follow this JSON format. Only include relevant columns.\n"


In [48]:
with open(csv_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="assistants")

# Step 2: Create Assistant with code interpreter access


# Step 3: Create a thread with your query


In [ ]:
client.beta.threads.messages.create

<bound method Messages.create of <openai.resources.beta.threads.messages.Messages object at 0x0000023C451872D0>>

In [ ]:
thread = client.beta.threads.messages.create(
    thread_id="thread_eCNLCmvP2TDfY0HxAWLeWnH1",
        
            role= "user",
            content= "Analyze this CSV file. Create 3 data visualizations and provide insight summaries.",
            attachments= [
                {
                    "file_id": uploaded_file.id,
                    "tools": [{"type": "code_interpreter"}]
                }
            ]
        
    
)


In [29]:
assistant_id = "asst_RhWhWWq7aN8OfQ8oP0cwAmcn"
thread_id="thread_eCNLCmvP2TDfY0HxAWLeWnH1"

In [ ]:

# Create a run
run = client.beta.threads.runs.create(
    thread_id="thread_eCNLCmvP2TDfY0HxAWLeWnH1",
    assistant_id=assistant_id
)

In [31]:
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
    if run_status.status in ["completed", "failed", "cancelled"]:
        break
    time.sleep(2)

'completed'

In [35]:


# Step 6: Get the response
messages = client.beta.threads.messages.list(thread_id=thread_id)
final_message = messages.data[0].content[0].text.value if messages.data else "No response."

print("=== Assistant's Output ===")
print(final_message)

IndexError: list index out of range

In [51]:
thread = client.beta.threads.create()

In [55]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    # assistant_id=assistant_id,
    role="user",
    content="Analyze this CSV file and summarize key trends.",
    attachments=[
        {
            "file_id": uploaded_file.id,
            "tools": [{"type": "code_interpreter"}]
        }
    ]
)

Message(id='msg_2MbBpRCeJiPlT3lHlzeznBY0', assistant_id=None, attachments=[Attachment(file_id='file-V4YS2UVv8KJ8UwG4J24Kyr', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Analyze this CSV file and summarize key trends.'), type='text')], created_at=1745102690, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_eJliMcrdCVdpSPUfPqHVBNNE')

In [43]:
uploaded_file.id

'file-XRehki1R5xS8NuiSwwMKv7'

In [56]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)

In [57]:
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status in ["completed", "failed", "cancelled"]:
        break
    time.sleep(2)

In [ ]:
# Step 6: Get the response
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status in ["completed", "failed", "cancelled"]:
        break
    time.sleep(2)
messages = client.beta.threads.messages.list(thread_id=thread.id)
final_message = messages.data[0].content[0].text.value if messages.data else "No response."

print("=== Assistant's Output ===")
print(final_message)

=== Assistant's Output ===
Here is the analysis of the categorical columns in the dataset:

```json
{
  "visualize": {
    "ethnicity": {
      "0": 41.63,
      "1": 19.41,
      "3": 17.16,
      "2": 14.46,
      "4": 7.34
    },
    "gender": {
      "0": 52.66,
      "1": 47.34
    }
  },
  "env": {
    "status": "Not equally distributed",
    "missing_or_underrepresented": ["ethnicity"]
  },
  "reason": {
    "verdict": "The dataset's ethnicity distribution is not even, with one category under 10% and another over 40%.",
    "suggestions": [
      "Increase representation of underrepresented ethnic categories.",
      "Ensure balanced representation across ethnic groups in new data collections."
    ]
  }
}
```

### Summary:
- **Ethnicity** is not equally distributed. The category '4' has only 7.34% representation, which is below 10%. On the other hand, category '0' is dominant with 41.63%.
- **Gender** is relatively balanced between the two categories (52.66% and 47.34%).

**Sug

In [59]:
import json
import re

In [61]:
raw_response = messages.data[0].content[0].text.value

# Extract content between triple backticks (```json ... ```)
match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", raw_response, re.DOTALL)

if match:
    json_block = match.group(1)  # this is the content inside ```
    try:
        parsed_json = json.loads(json_block)
        
        print(json.dumps(parsed_json, indent=2))
    except json.JSONDecodeError as e:
        print("❌ Failed to parse JSON:", e)
else:
    print("❌ No valid JSON block found between triple backticks.")

{
  "visualize": {
    "ethnicity": {
      "0": 41.63,
      "1": 19.41,
      "3": 17.16,
      "2": 14.46,
      "4": 7.34
    },
    "gender": {
      "0": 52.66,
      "1": 47.34
    }
  },
  "env": {
    "status": "Not equally distributed",
    "missing_or_underrepresented": [
      "ethnicity"
    ]
  },
  "reason": {
    "verdict": "The dataset's ethnicity distribution is not even, with one category under 10% and another over 40%.",
    "suggestions": [
      "Increase representation of underrepresented ethnic categories.",
      "Ensure balanced representation across ethnic groups in new data collections."
    ]
  }
}
